# **Public Transit Schedules**
This is the first notebook of Exploratory Data Analysis (EDA) on the data of public transit in Tenerife. I will analyse the bus routes, frequencies and timing, visualize them and compare the scheduled and real time performance.

Data sources:
 - GTFS Schedule: https://datos.tenerife.es/es/datos/conjuntos-de-datos/informacion-sobre-el-sistema-de-transporte-de-titsa-en-tenerife?return=aHR0cHM6Ly9kYXRvcy50ZW5lcmlmZS5lcy9lcy9kYXRvcy9jb25qdW50b3MtZGUtZGF0b3M=
 - GTFS Realtime: https://transitfeeds.com/p/transportes-interurbanos-de-tenerife/1058
 
Download date: 10 Sept 2023

Author: Miguel R. Alarcon

## Reading data


In [1]:
import pandas as pd
import os
import numpy as np

import sys
sys.path.append('..')
from utils.gtfs import *

Creating a dataframe with the stops info and frequency for every segment, i.e., every path joining one stop with other for a specific line id.

In [2]:
# Read the files from the data folders
Scheduled = GTFSReadFiles('../data/gtfs/scheduled')
Real = GTFSReadFiles('../data/gtfs/real')


In [3]:
dfs = Scheduled.dfs
Scheduled.info()


agency.txt
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   agency_id        1 non-null      int64 
 1   agency_name      1 non-null      object
 2   agency_url       1 non-null      object
 3   agency_timezone  1 non-null      object
 4   agency_lang      1 non-null      object
 5   agency_phone     1 non-null      object
dtypes: int64(1), object(5)
memory usage: 176.0+ bytes
None


stop_times.txt
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957059 entries, 0 to 957058
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   trip_id         957059 non-null  int64 
 1   arrival_time    957059 non-null  object
 2   departure_time  957059 non-null  object
 3   stop_id         957059 non-nul

In [7]:
class GTFSStops:
    
    def __init__(self, dataframes):
        self.dfs = dataframes
        
    def get_positions(self):
        positions_data = self.dfs["stops"][["stop_id", "stop_name", "stop_lat", "stop_lon"]]
        return positions_data
    
    def get_lines(self, stop_id):
        lines = {'route_id': [], 'route_name': [], 'previous_stop': [], 'next_stop': []}
        for trip in self.dfs["stop_times"][self.dfs["stop_times"]["stop_id"] == stop_id].trip_id.values:
            rid = self.dfs["trips"][self.dfs["trips"]["trip_id"] == trip].route_id.values[0]
            if rid not in lines['route_id']:
                lines['route_id'].append(rid)
                rnam = self.dfs["routes"][self.dfs["routes"]["route_id"] == rid].route_short_name.values[0]
                lines['route_name'].append(rnam)
                rst = self.dfs["stop_times"]
                stop_seq_id = rst.loc[(rst["trip_id"] == trip) & (rst["stop_id"] == stop_id)].stop_sequence.values[0]
                prev_stop = rst.loc[(rst["trip_id"] == trip) & (rst["stop_sequence"] == stop_seq_id - 1)].stop_id.values[0]
                next_stop = rst.loc[(rst["trip_id"] == trip) & (rst["stop_sequence"] == stop_seq_id + 1)].stop_id.values[0]
                lines['previous_stop'].append(prev_stop)
                lines['next_stop'].append(next_stop)

        df = pd.DataFrame(lines)
        return df
    
    def get_plot_df(self):
        df = self.get_positions()
        for i in df.index:
            lines = self.get_lines(df.loc[i, 'stop_id'])
            df.iloc[i, 'lines'] = lines['route_name'].values
        return df

In [8]:
Stops = GTFSStops(dfs)
Stops.get_positions()



,stop_id,stop_name,stop_lat,stop_lon
0,1100,ACORÁN,28.4184,-16.3038
1,1101,ATABARA,28.4176,-16.3067
2,1102,AYOZE,28.4163,-16.3088
3,1103,ADONAI,28.4152,-16.3113
4,1104,POLÍGONO LA CAMPANA,28.4088,-16.3176
...,...,...,...,...
3774,9479,AMÉRICA GONZÁLEZ,28.4803,-16.2564
3775,9481,LORENZO DE TOLOSA,28.4694,-16.2787
3776,9483,PASEO LA CONCORDIA,28.4648,-16.2522
3777,9484,LOS CAMINEROS,28.4952,-16.2163


In [9]:
Stops.get_lines(1312)

,route_id,route_name,previous_stop,next_stop
0,21,50,1311,1313
1,22,51,1311,1926
2,33,105,1311,1313


In [16]:
lines

,route_id,route_name,previous_stop,next_stop
0,45,138,1767,1101
1,1,139,1767,1101
2,4,142,1767,1101
3,50,935,1767,1101
4,2,940,1767,1101
5,3,941,1767,1101


In [17]:
lines['route_name'].values

array([138, 139, 142, 935, 940, 941])

In [27]:
Stops.get_lines(df.loc[i, 'stop_id'])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [23]:
df = Stops.get_positions()
for i in df.index:
    lines = Stops.get_lines(df.loc[i, 'stop_id'])
    df.loc[i, 'lines'] = (', ').join(lines['route_name'].values.astype(str))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [10]:
df = Stops.get_plot_df()
df

ValueError: Must have equal len keys and value when setting with an iterable

In [19]:
dfs['trips'].loc[dfs['trips']['route_id'] == 105]

,route_id,service_id,trip_id,trip_headsign,shape_id
7064,105,194453,6304066,ESTACIÓN LA OROTAVA (T),410
7065,105,194453,6304067,ESTACIÓN LA OROTAVA (T),410
7066,105,194453,6304068,ESTACIÓN LA OROTAVA (T),410
7067,105,194453,6304069,ESTACIÓN LA OROTAVA (T),410
7068,105,194453,6304070,ESTACIÓN LA OROTAVA (T),410
...,...,...,...,...,...
25173,105,198488,6447039,LAS DEHESAS (T),409
25174,105,198488,6447040,LAS DEHESAS (T),409
25175,105,198488,6447041,LAS DEHESAS (T),409
25176,105,198488,6447042,LAS DEHESAS (T),409


In [20]:
dfs['routes']

,route_id,agency_id,route_short_name,route_long_name,route_type,route_url,route_color,route_text_color
0,247,2,10,SANTA CRUZ - AEROPUERTO TENERIFE SUR,3,http://www.titsa.com/index.php/tus-guaguas/lin...,75AD1C,000000
1,13,2,11,LA LAGUNA -> EL SAUZAL- POR C/EL CALVARIO DE T...,3,http://www.titsa.com/index.php/tus-guaguas/lin...,75AD1C,000000
2,14,2,12,LA LAGUNA -> EL SAUZAL - POR ZONA CENTRO DE TA...,3,http://www.titsa.com/index.php/tus-guaguas/lin...,75AD1C,000000
3,15,2,14,SANTA CRUZ -> LA LAGUNA(POR LA CUESTA),3,http://www.titsa.com/index.php/tus-guaguas/lin...,75AD1C,000000
4,16,2,15,SANTA CRUZ - LA LAGUNA(Express),3,http://www.titsa.com/index.php/tus-guaguas/lin...,75AD1C,000000
...,...,...,...,...,...,...,...,...
168,251,2,970,INTERCAMBIADOR - SAN ANDRÉS,3,http://www.titsa.com/index.php/tus-guaguas/lin...,3F8FCF,000000
169,149,2,971,INTERCAMBIADOR - BARRIO SALUD - OFRA (NOCTURNA),3,http://www.titsa.com/index.php/tus-guaguas/lin...,3F8FCF,000000
170,150,2,972,INTERCAMBIADOR - B. SALUD (CUESTA PIEDRA) (NOC...,3,http://www.titsa.com/index.php/tus-guaguas/lin...,3F8FCF,000000
171,238,2,974,SANTA CRUZ - SANTA MARÍA - AÑAZA (NOCTURNO),3,http://www.titsa.com/index.php/tus-guaguas/lin...,3F8FCF,000000


In [12]:
stops = Stops(Scheduled.dataframes)
dfp = stops.get_positions()
dfp.loc[dfp.stop_id == 1312]

,stop_id,stop_name,stop_lat,stop_lon
181,1312,TOMÁS GONZÁLEZ RIVERO,28.5342,-16.3606


In [17]:
stops.get_lines(1312)

,route_id,previous_stop,next_stop
0,21,1311,1313
1,22,1311,1926
2,33,1311,1313


In [ ]:
Quiero tambien una función llamada get_times que dado el stop_id devuelva un df con route_id, date, time, donde date lo obtiene de calendar_dates. Llamando a esta, quiero otra función llamada get_bus_count que, dado el stop_id, date_from y date_to, de el numero de veces que un bus de cada una de las lineas pasa en el intervalo seleccionado. Si date_from y date_to está fuera del intervalo disponible, prieta un warning con los intervalos disponibles.

In [59]:
stop_times_df.loc[stop_times_df['trip_id'] == 6289828]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_id_next
0,6289828,05:40:00,05:40:00,9181,1,7118.0
1,6289828,05:51:00,05:51:00,7118,2,7134.0
2,6289828,06:17:27,06:17:27,7134,3,8185.0
3,6289828,06:21:05,06:21:05,8185,4,8326.0
4,6289828,06:24:20,06:24:20,8326,5,7571.0
5,6289828,06:25:36,06:25:36,7571,6,NaN


In [63]:
stop_times_df.loc[stop_times_df['service_id'] == 6413738]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_id_next
418359,6413738,17:05:00,17:05:00,4358,1,4360.0
418360,6413738,17:05:51,17:05:51,4360,2,4359.0
418361,6413738,17:06:29,17:06:29,4359,3,4361.0
418362,6413738,17:08:05,17:08:05,4361,4,4362.0
418363,6413738,17:08:46,17:08:46,4362,5,4363.0
418364,6413738,17:09:23,17:09:23,4363,6,4364.0
418365,6413738,17:09:57,17:09:57,4364,7,4365.0
418366,6413738,17:10:41,17:10:41,4365,8,4366.0
418367,6413738,17:11:45,17:11:45,4366,9,4367.0
418368,6413738,17:12:39,17:12:39,4367,10,4368.0


In [66]:
# Merge trips_df and calendar_dates_df on service_id to obtain a relation between trip_id and date
trip_dates_simple_df = trips_df[['trip_id', 'service_id']].merge(calendar_dates_df[['service_id', 'date']], on='service_id', how='left')

# Drop the service_id column as it's no longer needed
trip_dates_simple_df = trip_dates_simple_df.drop(columns=['service_id'])

# Display the resulting dataframe
trip_dates_simple_df


,trip_id,date
0,6289828,20230917
1,6289828,20230924
2,6289828,20231001
3,6289828,20231008
4,6289828,20231012
...,...,...
928358,6456126,20230831
928359,6456126,20230904
928360,6456126,20230905
928361,6456126,20230906


In [64]:
trips_df.loc[trips_df['service_id'] == 197466]

,route_id,service_id,trip_id,trip_headsign,shape_id
15087,128,197466,6413728,ESTACIÓN ICOD (T),2383
15088,128,197466,6413729,ESTACIÓN ICOD (T),2383
15089,128,197466,6413730,ESTACIÓN ICOD (T),2383
15090,128,197466,6413731,ESTACIÓN ICOD (T),2383
15091,128,197466,6413732,ESTACIÓN ICOD (T),2383
15092,128,197466,6413733,ESTACIÓN ICOD (T),2383
15093,128,197466,6413734,ESTACIÓN ICOD (T),2383
15094,128,197466,6413735,ESTACIÓN ICOD (T),2383
15095,128,197466,6413736,ESTACIÓN ICOD (T),2383
15096,128,197466,6413737,ESTACIÓN ICOD (T),2383


In [50]:
# Read the GTFS files into dataframes
files_path = '../data/gtfs/planned/GTFS_Expected'
stop_times_df = pd.read_csv(os.path.join(files_path, 'stop_times.txt'))
stops_df = pd.read_csv(os.path.join(files_path, 'stops.txt'))
trips_df = pd.read_csv(os.path.join(files_path, 'trips.txt'))
routes_df = pd.read_csv(os.path.join(files_path, 'routes.txt'))
calendar_dates_df = pd.read_csv(os.path.join(files_path, 'calendar_dates.txt'))

# Create segments
stop_times_df['stop_id_next'] = stop_times_df.groupby('trip_id')['stop_id'].shift(-1)
segments_df = stop_times_df.dropna(subset=['stop_id_next'])
segments_df['stop_id_next'] = segments_df['stop_id_next'].astype(int)
segments_df = segments_df.merge(stops_df[['stop_id', 'stop_lat', 'stop_lon']], on='stop_id')
segments_df = segments_df.merge(stops_df[['stop_id', 'stop_lat', 'stop_lon']],
                                left_on='stop_id_next', right_on='stop_id', suffixes=('', '_next'))

segments_df = segments_df.merge(trips_df, on='trip_id', how='left')
segments_df = segments_df.merge(calendar_dates_df, on='service_id', how='left')


#segments_df = segments_df.drop(columns=['stop_id_next', 'stop_sequence'])
segments_df['frequency'] = segments_df.groupby(['stop_id', 'stop_lat_next', 'stop_lon_next']).transform('count')['trip_id']

segments_df = segments_df.loc[:,~segments_df.columns.duplicated()].copy()
segments_df = segments_df.drop(columns=['departure_time', 'stop_sequence'])


# Now merge segments_df with trip_dates_df on trip_id to add the date column
segments_df = segments_df.merge(trips_df, on='trip_id', how='left')




print(segments_df.info())
segments_df.head()


KeyboardInterrupt: 

In [69]:
segments_df

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_id_next,stop_lat,stop_lon,stop_id_next,stop_lat_next,stop_lon_next,route_id,service_id,trip_headsign,shape_id,date,exception_type
0,6289828,05:40:00,05:40:00,9181,1,7118,28.4590,-16.2527,7118,28.382,-16.3634,247,193969,AEROPUERTO TENERIFE SUR (T),2640,20230917,1
1,6289828,05:40:00,05:40:00,9181,1,7118,28.4590,-16.2527,7118,28.382,-16.3634,247,193969,AEROPUERTO TENERIFE SUR (T),2640,20230924,1
2,6289828,05:40:00,05:40:00,9181,1,7118,28.4590,-16.2527,7118,28.382,-16.3634,247,193969,AEROPUERTO TENERIFE SUR (T),2640,20231001,1
3,6289828,05:40:00,05:40:00,9181,1,7118,28.4590,-16.2527,7118,28.382,-16.3634,247,193969,AEROPUERTO TENERIFE SUR (T),2640,20231008,1
4,6289828,05:40:00,05:40:00,9181,1,7118,28.4590,-16.2527,7118,28.382,-16.3634,247,193969,AEROPUERTO TENERIFE SUR (T),2640,20231012,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25356462,6455984,16:45:00,16:45:00,9478,1,9477,28.5739,-16.1878,9477,28.572,-16.1923,239,198735,BAJO ASANO,2323,20230909,1
25356463,6455985,18:00:00,18:00:00,9478,1,9477,28.5739,-16.1878,9477,28.572,-16.1923,239,198735,BAJO ASANO,2323,20230902,1
25356464,6455985,18:00:00,18:00:00,9478,1,9477,28.5739,-16.1878,9477,28.572,-16.1923,239,198735,BAJO ASANO,2323,20230909,1
25356465,6455986,18:50:00,18:50:00,9478,1,9477,28.5739,-16.1878,9477,28.572,-16.1923,239,198735,BAJO ASANO,2323,20230902,1


In [12]:
# Compute the frequency for each segment
segments_df['frequency'] = segments_df.groupby(['stop_id', 'stop_lat_next', 'stop_lon_next']).transform('count')['trip_id']

# Display the segments dataframe with frequency
segments_df.head()


INFO:root:Reading "routes.txt".
INFO:root:Reading "stops.txt".
INFO:root:Reading "shapes.txt".
INFO:root:Reading "stop_times.txt".
INFO:root:get trips in stop_times
INFO:root:accessing trips
INFO:root:Reading "trips.txt".
INFO:root:File "calendar.txt" not found.
INFO:root:Reading "calendar_dates.txt".


AttributeError: 'DataFrame' object has no attribute 'start_date'

In [13]:

# Load an empty map
from keplergl import KeplerGl
map_1 = KeplerGl()
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl()

In [10]:
routes

,route_id,agency_id,route_short_name,route_long_name,route_type,route_url,route_color,route_text_color
0,247,2,10,SANTA CRUZ - AEROPUERTO TENERIFE SUR,3,http://www.titsa.com/index.php/tus-guaguas/lin...,75AD1C,000000
1,13,2,11,LA LAGUNA -> EL SAUZAL- POR C/EL CALVARIO DE T...,3,http://www.titsa.com/index.php/tus-guaguas/lin...,75AD1C,000000
2,14,2,12,LA LAGUNA -> EL SAUZAL - POR ZONA CENTRO DE TA...,3,http://www.titsa.com/index.php/tus-guaguas/lin...,75AD1C,000000
3,15,2,14,SANTA CRUZ -> LA LAGUNA(POR LA CUESTA),3,http://www.titsa.com/index.php/tus-guaguas/lin...,75AD1C,000000
4,16,2,15,SANTA CRUZ - LA LAGUNA(Express),3,http://www.titsa.com/index.php/tus-guaguas/lin...,75AD1C,000000
...,...,...,...,...,...,...,...,...
168,251,2,970,INTERCAMBIADOR - SAN ANDRÉS,3,http://www.titsa.com/index.php/tus-guaguas/lin...,3F8FCF,000000
169,149,2,971,INTERCAMBIADOR - BARRIO SALUD - OFRA (NOCTURNA),3,http://www.titsa.com/index.php/tus-guaguas/lin...,3F8FCF,000000
170,150,2,972,INTERCAMBIADOR - B. SALUD (CUESTA PIEDRA) (NOC...,3,http://www.titsa.com/index.php/tus-guaguas/lin...,3F8FCF,000000
171,238,2,974,SANTA CRUZ - SANTA MARÍA - AÑAZA (NOCTURNO),3,http://www.titsa.com/index.php/tus-guaguas/lin...,3F8FCF,000000
